In [1]:
# import necessary package
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import pdb
import pickle, os
from torch_geometric.utils import dense_to_sparse, f1_score, accuracy
from torch_geometric.data import Data
import torch_sparse
import pickle
from scipy.sparse import csr_matrix
import scipy.sparse as sp

# import files
from parseargs import *
args.dataset='ACM'

In [2]:
# load data
dataroot = 'data'
with open(os.path.join(dataroot, args.dataset, 'node_features.pkl'),'rb') as f:
    node_features = pickle.load(f)
with open(os.path.join(dataroot, args.dataset, 'edges.pkl'),'rb') as f:
    edges = pickle.load(f)
with open(os.path.join(dataroot, args.dataset, 'labels.pkl'),'rb') as f:
    labels = pickle.load(f)

/tmp/ipykernel_886162/2204756402.py:6: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  edges = pickle.load(f)
/tmp/ipykernel_886162/2204756402.py:6: DeprecationWarning: Please use `csc_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csc` namespace is deprecated.
  edges = pickle.load(f)


In [3]:
# data prepare
num_nodes = edges[0].shape[0]
for i,edge in enumerate(edges):
    if i ==0:
        A = torch.from_numpy(edge.todense()).type(torch.FloatTensor).unsqueeze(-1)
    else:
        A = torch.cat([A,torch.from_numpy(edge.todense()).type(torch.FloatTensor).unsqueeze(-1)], dim=-1)
A = torch.cat([A,torch.eye(num_nodes).type(torch.FloatTensor).unsqueeze(-1)], dim=-1)
A = A.cuda()

node_features = torch.from_numpy(node_features).type(torch.cuda.FloatTensor)
train_node = torch.from_numpy(np.array(labels[0])[:,0]).type(torch.cuda.LongTensor)
train_target = torch.from_numpy(np.array(labels[0])[:,1]).type(torch.cuda.LongTensor)

valid_node = torch.from_numpy(np.array(labels[1])[:,0]).type(torch.cuda.LongTensor)
valid_target = torch.from_numpy(np.array(labels[1])[:,1]).type(torch.cuda.LongTensor)
test_node = torch.from_numpy(np.array(labels[2])[:,0]).type(torch.cuda.LongTensor)
test_target = torch.from_numpy(np.array(labels[2])[:,1]).type(torch.cuda.LongTensor)


num_classes = torch.max(train_target).item()+1

In [4]:
# model define
from model import GTN
model = GTN(num_edge=A.shape[-1], num_channels=args.num_channels, w_in = node_features.shape[1], w_out = args.node_dim, num_class=num_classes, num_layers=args.num_layers, norm=args.norm)
model = model.cuda()

if args.adaptive_lr == 'false':
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)
else:
    optimizer = torch.optim.Adam([{'params':model.weight},
                                {'params':model.linear1.parameters()},
                                {'params':model.linear2.parameters()},
                                {"params":model.layers.parameters(), "lr":0.5}
                                ], lr=0.005, weight_decay=0.001)

loss = nn.CrossEntropyLoss()
best_val_loss = 10000
best_test_loss = 10000
best_train_loss = 10000
best_train_f1 = 0
best_val_f1 = 0
best_test_f1 = 0

In [5]:
for i in range(args.epoch):
    for param_group in optimizer.param_groups:
        if param_group['lr'] > 0.005:
            param_group['lr'] = param_group['lr'] * 0.9
    print('Epoch:  ',i+1)
    model.zero_grad()
    # ====================
    model.train()
    # print(A.device, node_features.device, valid_node.device, valid_target.device)
    loss,y_train,Ws = model(A, node_features, train_node, train_target)
    train_f1 = torch.mean(f1_score(torch.argmax(y_train.detach(),dim=1), train_target, num_classes=num_classes)).cpu().numpy()
    print('Train - Loss: {}, Macro_F1: {}'.format(loss.detach().cpu().numpy(), train_f1))
    loss.backward()
    optimizer.step()
    # ====================
    model.eval()
    with torch.no_grad():
        val_loss, y_valid,_ = model.forward(A, node_features, valid_node, valid_target)
        val_f1 = torch.mean(f1_score(torch.argmax(y_valid,dim=1), valid_target, num_classes=num_classes)).cpu().numpy()
        print('Valid - Loss: {}, Macro_F1: {}'.format(val_loss.detach().cpu().numpy(), val_f1))
        test_loss, y_test,W = model.forward(A, node_features, test_node, test_target)
        test_f1 = torch.mean(f1_score(torch.argmax(y_test,dim=1), test_target, num_classes=num_classes)).cpu().numpy()
        print('Test - Loss: {}, Macro_F1: {}\n'.format(test_loss.detach().cpu().numpy(), test_f1))
    if val_f1 > best_val_f1:
        best_val_loss = val_loss.detach().cpu().numpy()
        best_test_loss = test_loss.detach().cpu().numpy()
        best_train_loss = loss.detach().cpu().numpy()
        best_train_f1 = train_f1
        best_val_f1 = val_f1
        best_test_f1 = test_f1 
print('---------------Best Results--------------------')
print('Train - Loss: {}, Macro_F1: {}'.format(best_train_loss, best_train_f1))
print('Valid - Loss: {}, Macro_F1: {}'.format(best_val_loss, best_val_f1))
print('Test - Loss: {}, Macro_F1: {}'.format(best_test_loss, best_test_f1))


Epoch:   1
Train - Loss: 1.1026039123535156, Macro_F1: 0.2510066032409668
Valid - Loss: 1.064597725868225, Macro_F1: 0.3927691876888275
Test - Loss: 1.064851999282837, Macro_F1: 0.42511463165283203

Epoch:   2
Train - Loss: 1.0621116161346436, Macro_F1: 0.44511276483535767
Valid - Loss: 1.0005815029144287, Macro_F1: 0.4575720429420471
Test - Loss: 1.0020924806594849, Macro_F1: 0.42890986800193787

Epoch:   3
Train - Loss: 0.9887065887451172, Macro_F1: 0.48181185126304626
Valid - Loss: 0.9127394556999207, Macro_F1: 0.5152896642684937
Test - Loss: 0.9191950559616089, Macro_F1: 0.4987413287162781

Epoch:   4
Train - Loss: 0.8885777592658997, Macro_F1: 0.5382675528526306
Valid - Loss: 0.7590799331665039, Macro_F1: 0.8626084327697754
Test - Loss: 0.7673485279083252, Macro_F1: 0.855971097946167

Epoch:   5
Train - Loss: 0.7249962687492371, Macro_F1: 0.9156365394592285
Valid - Loss: 0.6255304217338562, Macro_F1: 0.9033226370811462
Test - Loss: 0.6377333998680115, Macro_F1: 0.8835882544517517
